In [ ]:
!pip install openai
!pip install -q transformers accelerate sentencepiece gradio
!pip install sentencepiece
!pip install backoff

In [ ]:
# mount google drive
from google.colab import drive
import sys

drive.mount('/content/drive')
sys.path.append(<PATH_TO_DIRECTORY>)
%cd <PATH_TO_DIRECTORY>

In [ ]:
import pandas as pd
from math import isnan
import copy
import csv
import re
import sys
import os
import json
import backoff
import json



In [ ]:
from openai import OpenAI
import openai

client = OpenAI(
    api_key = <OPEN_AI_API_KEY>
    )

@backoff.on_exception(backoff.expo, (openai.RateLimitError, openai.APIError, openai.InternalServerError)) #openai.ServiceUnavailableError,

def chat_completions_with_backoff(**kwargs):
  return client.chat.completions.create(**kwargs)

In [ ]:
def is_nan(x):
    return isinstance(x, float) and isnan(x)

In [ ]:
default_prompt ="""Rewrite the original text changing all gendered pronouns and nouns referencing people to the opposite gender. Examples of gendered nouns include princess, seamstress, and bride. Do not modify adjectives or verbs. Do not include any explanations.

original: "there once lived a poor widow who supported herself and her only son by gleaning in the fields the stalks of grain that had been missed by the reapers . he had big blue eyes , and fair golden curls , and he loved his good mother very dearly , and was never more pleased than when she allowed him to help her with her work ."
rewritten: "there once lived a poor widower who supported himself and his only daughter by gleaning in the fields the stalks of grain that had been missed by the reapers . she had big blue eyes , and fair golden curls , and she loved her good father very dearly , and was never more pleased than when he allowed her to help him with his work ."

original: "how did the poor widow support herself and her son ?"
rewritten: "how did the poor widower support himself and his daughter?"

original: "gleaning in the fields the stalks of grain that had been missed by the reapers ."
rewritten: "gleaning in the fields the stalks of grain that had been missed by the reapers ."

"""


In [ ]:
# uses https://github.com/uclanlp/corefBias/blob/master/WinoBias/wino/word_swapper.py
# assumes generalized_swaps.txt and extra_gendered_words.txt are in the subdirectory ./pertubation_dicts

def word_swapper(input, output):

  data = pd.read_csv(input)
  counterfactual_data = data.copy(deep=True)
  count = 0
  text_columns = [col for col in counterfactual_data.columns if (col in ['text', 'question'] or col.startswith('answer'))]


  for index, row in counterfactual_data.iterrows():
    row_columns = [col for col in text_columns if not is_nan(row[col])]
    for col in row_columns:
      prompt = '{}\noriginal: {}\nrewritten: '.format(default_prompt, row[col])
      response = chat_completions_with_backoff(model="gpt-3.5-turbo", messages=[{"role": "user", "content":prompt}])
      perturbed_text = response.choices[0].message.content
      counterfactual_data.at[index, col] = perturbed_text

    count += 1
  print(count)
  counterfactual_fairytale_data.to_csv(output, index=False)
  return counterfactual_data



In [ ]:
counterfactual_fairytale_data = word_swapper('./fairytaleqa_preprocessed/fairytaleqa_preprocessed_test.csv', './llm_rewrite/fairytaleqa_preprocessed_test.csv')